In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-04-06 03:49:30,658 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.2.0.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/working


In [6]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()}

graphnet: INFO     2023-04-06 03:49:33 - get_logger - Writing log to logs/graphnet_20230406-034933.log
graphnet: WARNING  2023-04-06 03:49:33 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-04-06 03:49:34.563230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 03:49:34.736533: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different nume

In [7]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-04-06 04:03:40,821 [INFO][utils] Output directory: output/2023-04-06_04-03-40
2023-04-06 04:03:40,821 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-04-06 04:03:40,821 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-04-06 04:03:40,821 [INFO][utils] Fix seed: 440
2023-04-06 04:03:40,822 [INFO][15-validation] Validation stage 1.
2023-04-06 04:03:41,195 [INFO][15-validation] Base score: 1.0141627366718835
2023-04-06 04:03:41,207 [INFO][15-validation] Num of low sigma events: 52483, Num of high sigma events: 147517
2023-04-06 04:03:41,255 [INFO][15-validation] Low sigma score: 0.2109929984713962, High sigma score: 1.29991120884781
2023-04-06 04:03:41,255 [INFO][15-validation] Done.


In [8]:
!mv results.csv results_stage2.csv

In [9]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()} inference_params.model_path=icecube/model_1_100.pth global_params.seed=1440

graphnet: INFO     2023-04-06 04:03:44 - get_logger - Writing log to logs/graphnet_20230406-040344.log
graphnet: WARNING  2023-04-06 04:03:44 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-04-06 04:03:45.866696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 04:03:46.036589: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different nume

In [10]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-04-06 04:17:59,942 [INFO][utils] Output directory: output/2023-04-06_04-17-59
2023-04-06 04:17:59,942 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-04-06 04:17:59,942 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-04-06 04:17:59,942 [INFO][utils] Fix seed: 440
2023-04-06 04:17:59,944 [INFO][15-validation] Validation stage 1.
2023-04-06 04:18:00,320 [INFO][15-validation] Base score: 1.0140286220873347
2023-04-06 04:18:00,331 [INFO][15-validation] Num of low sigma events: 51159, Num of high sigma events: 148841
2023-04-06 04:18:00,379 [INFO][15-validation] Low sigma score: 0.2067213735405639, High sigma score: 1.291512860492104
2023-04-06 04:18:00,379 [INFO][15-validation] Done.


In [11]:
!python 20-ensemble.py settings.in_kaggle={in_kaggle()}

2023-04-06 04:18:03,541 [INFO][utils] Output directory: output/2023-04-06_04-18-03
2023-04-06 04:18:03,541 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-04-06 04:18:03,541 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-04-06 04:18:03,541 [INFO][utils] Fix seed: 440
2023-04-06 04:18:04,040 [INFO][20-ensemble] Results has same index.
2023-04-06 04:18:04,818 [INFO][20-ensemble] Base score: 1.018052678751734
2023-04-06 04:18:04,844 [INFO][20-ensemble] Num of low sigma events: 51476, Num of high sigma events: 148524
2023-04-06 04:18:04,890 [INFO][20-ensemble] Low sigma score: 0.21027608622758714, High sigma score: 1.2980148927829545
2023-04-06 04:18:04,890 [INFO][20-ensemble] Done.
